# APS 3 - Aula 20

A funcao 'plota' produz um gráfico da estrutura definida pela matriz de nos N 
e pela incidencia Inc.

Sugestao de uso:

from funcoesTermosol import plota
plota(N,Inc)
-------------------------------------------------------------------------------
A funcao 'importa' retorna o numero de nos [nn], a matriz dos nos [N], o numero
de membros [nm], a matriz de incidencia [Inc], o numero de cargas [nc], o vetor
carregamento [F], o numero de restricoes [nr] e o vetor de restricoes [R] 
contidos no arquivo de entrada.

Sugestao de uso:
    
from funcoesTermosol import importa
[nn,N,nm,Inc,nc,F,nr,R] = importa('entrada.xlsx')
-------------------------------------------------------------------------------
A funcao 'geraSaida' cria um arquivo nome.txt contendo as reacoes de apoio Ft, 
deslocamentos Ut, forcas Fi e tensoes Ti internas. As entradas devem ser 
vetores coluna.

Sugestao de uso:
    
from funcoesTermosol import geraSaida
geraSaida(nome,Ft,Ut,Epsi,Fi,Ti)
-------------------------------------------------------------------------------

In [1]:
from cmath import *
from math import *
from numpy import linalg
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt

## Funções:

In [2]:
def plota(N,Inc):
    # Numero de membros
    nm = len(Inc[:,0])
    
    import matplotlib as mpl
    import matplotlib.pyplot as plt

#    plt.show()
    fig = plt.figure()
    # Passa por todos os membros
    for i in range(nm):
        
        # encontra no inicial [n1] e final [n2] 
        n1 = int(Inc[i,0])
        n2 = int(Inc[i,1])        

        plt.plot([N[0,n1-1],N[0,n2-1]],[N[1,n1-1],N[1,n2-1]],color='r',linewidth=3)


    plt.xlabel('x [m]')
    plt.ylabel('y [m]')
    plt.grid(True)
    plt.axis('equal')
    plt.show()
    

In [3]:
def importa(entradaNome):
    
    import numpy as np
    import xlrd
    
    arquivo = xlrd.open_workbook(entradaNome)
    
    ################################################## Ler os nos
    nos = arquivo.sheet_by_name('Nos')
    
    # Numero de nos
    nn = int(nos.cell(1,3).value)
                 
    # Matriz dos nós
    N = np.zeros((2,nn))
    
    for c in range(nn):
        N[0,c] = nos.cell(c+1,0).value
        N[1,c] = nos.cell(c+1,1).value
    
    ################################################## Ler a incidencia
    incid = arquivo.sheet_by_name('Incidencia')
    
    # Numero de membros
    nm = int(incid.cell(1,5).value)
                 
    # Matriz de incidencia
    Inc = np.zeros((nm,4))
    
    for c in range(nm):
        Inc[c,0] = int(incid.cell(c+1,0).value)
        Inc[c,1] = int(incid.cell(c+1,1).value)
        Inc[c,2] = incid.cell(c+1,2).value
        Inc[c,3] = incid.cell(c+1,3).value
    
    ################################################## Ler as cargas
    carg = arquivo.sheet_by_name('Carregamento')
    
    # Numero de cargas
    nc = int(carg.cell(1,4).value)
                 
    # Vetor carregamento
    F = np.zeros((nn*2,1))
    
    for c in range(nc):
        no = carg.cell(c+1,0).value
        xouy = carg.cell(c+1,1).value
        GDL = int(no*2-(2-xouy)) 
        F[GDL-1,0] = carg.cell(c+1,2).value
         
    ################################################## Ler restricoes
    restr = arquivo.sheet_by_name('Restricao')
    
    # Numero de restricoes
    nr = int(restr.cell(1,3).value)
                 
    # Vetor com os graus de liberdade restritos
    R = np.zeros((nr,1))
    
    for c in range(nr):
        no = restr.cell(c+1,0).value
        xouy = restr.cell(c+1,1).value
        GDL = no*2-(2-xouy) 
        R[c,0] = GDL-1


    return nn,N,nm,Inc,nc,F,nr,R

In [4]:
def geraSaida(nome,Ft,Ut,Epsi,Fi,Ti):
    nome = nome + '.txt'
    f = open(nome,"w+")
    f.write('Reacoes de apoio [N]\n')
    f.write(str(Ft))
    f.write('\n\nDeslocamentos [m]\n')
    f.write(str(Ut))
    f.write('\n\nDeformacoes []\n')
    f.write(str(Epsi))
    f.write('\n\nForcas internas [N]\n')
    f.write(str(Fi))
    f.write('\n\nTensoes internas [Pa]\n')
    f.write(str(Ti))
    f.close()

In [5]:
num_nos, matriz_nos, num_membros, matriz_incidencia, num_cargas, v_carregamento, num_rest, v_rest = importa("entrada.xlsx")

In [6]:
# Numero de nós
print(num_nos)

3


In [7]:
# Matriz de nós
print(matriz_nos)

[[0.  0.  0.3]
 [0.  0.4 0.4]]


In [8]:
# Número de membros
print(num_membros)

3


In [9]:
# Matriz de incidencia
print(matriz_incidencia)

[[1.0e+00 2.0e+00 2.1e+11 2.0e-04]
 [2.0e+00 3.0e+00 2.1e+11 2.0e-04]
 [3.0e+00 1.0e+00 2.1e+11 2.0e-04]]


In [10]:
# Número de cargas
print(num_cargas)

2


In [11]:
# Vetor Carregamento
print(v_carregamento)

[[   0.]
 [   0.]
 [   0.]
 [   0.]
 [ 150.]
 [-100.]]


In [12]:
# Número de restrições
print(num_rest)

3


In [13]:
# Vetor de restrições
print(v_rest)

[[0.]
 [2.]
 [3.]]


In [14]:
# Função para achar a matriz de conectividade de CADA elemento

def matriz_conectvidade(numero_do_membro):
    conectividade = num_membros*[0]
    
    # Lógica a partir da matriz de incidencia
    # O numero do membro-1 é a linha a matriz que eu tenho que utilizar
    no_1 = int(matriz_incidencia[numero_do_membro-1, 0])
    no_2 = int(matriz_incidencia[numero_do_membro-1, 1])
    
   
    conectividade[no_1-1]= -1
    conectividade[no_2-1]= 1
    
    
    
    return conectividade
    
    
    
    

In [15]:
# TESTANDO FUNÇÃO ACIMA
print(matriz_conectvidade(1))
#print(matriz_conectvidade(2))
#print(matriz_conectvidade(3))

[-1, 1, 0]


In [16]:
# Função para obter a Matriz de conectividade transposta

def conec_global_T():
    C = []
    i=1
    # Achar cada vetor de conectividade
    # e ir concatenando
    while(i<=num_nos):
        Ce = matriz_conectvidade(i)
        C.append(Ce)
        
        i+=1
        
    Conc = np.array(C)
    
    Conc_T = np.transpose(Conc)
        
    return Conc_T
    
    
    
    
    

In [17]:
C_t=conec_global_T()
print(C_t)

[[-1  0  1]
 [ 1 -1  0]
 [ 0  1 -1]]


In [18]:
# Funcção para obter a matriz dos membros

def matriz_membros(mat_conc_T):
    print(matriz_nos)
    matriz_membros= np.dot(matriz_nos, mat_conc_T)
    
    return matriz_membros
    
    

In [19]:
matriz_dos_membros = matriz_membros(C_t)
print(matriz_dos_membros)

[[0.  0.  0.3]
 [0.  0.4 0.4]]
[[ 0.   0.3 -0.3]
 [ 0.4  0.  -0.4]]


In [20]:
# Função para achar S de cada elemento

def S_e(n_membro):
    E =  matriz_incidencia[n_membro-1, 2]
    
    A = matriz_incidencia[n_membro-1, 3]
    
    
    no_1 = int(matriz_incidencia[n_membro-1, 0])
    no_2 = int(matriz_incidencia[n_membro-1, 1])
    
    # Pegar a coordenada do no_1
    x_no1 = matriz_nos[0, no_1-1]
    y_no1 = matriz_nos[1, no_1-1]
    
    # Pegar a coordenada do no_2
    x_no2 = matriz_nos[0, no_2-1]
    y_no2 = matriz_nos[1, no_2-1]
    
    l = sqrt((x_no2-x_no1)**2+(y_no2-y_no1)**2)
  
    
    primeira_parte = (E*A)/l
    
    
    
    lista_m = matriz_dos_membros[:, n_membro-1]
    
    
    matriz_lista_m = np.array([lista_m])
    
    
    matriz_lista_m_T = np.transpose(matriz_lista_m)
        
    
    me = 0
    i=0
    while(i<len(lista_m)):
        me+=(lista_m[i]**2)
        
        i+=1
        
    segunda_parte = (np.dot(matriz_lista_m_T, matriz_lista_m))/me
    
    
    
    SE = primeira_parte*segunda_parte
    
    return SE
        
    
    

In [21]:
S_e(3)

array([[30240000.        , 40320000.        ],
       [40320000.        , 53760000.00000001]])

In [22]:
# Função que calcula a Matriz de rigidez para cada elemento

def  K_e(n_membro):
    MC = np.array([matriz_conectvidade(n_membro)])
    MC_T = np.transpose(MC)
    
    
    Se = S_e(n_membro)
    
    dot = np.kron(MC, MC_T)
    
    
    
    Ke = np.kron(dot, Se)
    
    
    return Ke
    
    

In [23]:
Ke_1=K_e(1)
print(Ke_1)

[[ 0.00e+00  0.00e+00 -0.00e+00 -0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  1.05e+08 -0.00e+00 -1.05e+08  0.00e+00  0.00e+00]
 [-0.00e+00 -0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [-0.00e+00 -1.05e+08  0.00e+00  1.05e+08  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00]]


In [24]:
# Função para achar a matriz de rigidez global 

def matriz_global():
    i=1
    Kg = np.zeros((6, 6)) #sempre vai ser num_membros*2???
    while i<=num_membros:
        ke=K_e(i)
        
        Kg+=ke
        
        i+=1
        
    return Kg

In [25]:
KG = matriz_global()

In [26]:
print(KG)

[[ 3.0240e+07  4.0320e+07  0.0000e+00  0.0000e+00 -3.0240e+07 -4.0320e+07]
 [ 4.0320e+07  1.5876e+08  0.0000e+00 -1.0500e+08 -4.0320e+07 -5.3760e+07]
 [ 0.0000e+00  0.0000e+00  1.4000e+08  0.0000e+00 -1.4000e+08  0.0000e+00]
 [ 0.0000e+00 -1.0500e+08  0.0000e+00  1.0500e+08  0.0000e+00  0.0000e+00]
 [-3.0240e+07 -4.0320e+07 -1.4000e+08  0.0000e+00  1.7024e+08  4.0320e+07]
 [-4.0320e+07 -5.3760e+07  0.0000e+00  0.0000e+00  4.0320e+07  5.3760e+07]]


In [27]:
# Vetor globla de forças concentradas
# Pg=[R1x, 0, R2x, R2y, 150, -100]
# Condição de contorno
Pg=[0, 150, -100]
Pg=np.array([Pg])
Pg =  np.transpose(Pg)

In [40]:
# Aplicando as condições de contorno

def MR_para_soluca(matriz):
   
    matriz_certa = np.zeros((6, 6))
    
    # Apaga primeira, terceira e quarta linhas
    matriz_certa = np.delete(matriz, (0, 2, 3), axis=0)
    # Apaga primeira terceira e quarta colunas
    matriz_certa = np.delete(matriz_certa, (0, 2, 3), axis=1)
    
    
    
    return matriz_certa

In [41]:
KG_solucao=MR_para_soluca(KG)
print(KG_solucao)

[[ 1.5876e+08 -4.0320e+07 -5.3760e+07]
 [-4.0320e+07  1.7024e+08  4.0320e+07]
 [-5.3760e+07  4.0320e+07  5.3760e+07]]


In [30]:
# Descobrindo u2 u5 u6
import scipy.linalg
def calcula_deslocamentos(matriz_rigidez, matriz_força):
    
    L,U = scipy.linalg.lu(matriz_rigidez, permute_l=True)
    
    y = scipy.linalg.solve(L, matriz_força)
    
    x = scipy.linalg.solve_triangular(U, y)
    
    return x

In [31]:
u = calcula_deslocamentos(KG_solucao, Pg)
u

array([[-9.52380952e-07],
       [ 1.60714286e-06],
       [-4.01785714e-06]])

In [32]:
u_completo = np.zeros((6, 1))
u_completo[1, 0]=u[0,0]
u_completo[4, 0]=u[1,0]
u_completo[5, 0]=u[2,0]
u_completo

array([[ 0.00000000e+00],
       [-9.52380952e-07],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 1.60714286e-06],
       [-4.01785714e-06]])

In [33]:
e=1
R1x=0
while e<=6:
    R1x+=KG[0, e-1]*u_completo[e-1, 0]
    
    e+=1
print(R1x)

74.99999999999997


In [34]:
e=1
R2x=0
while e<=6:
    R2x+=KG[2, e-1]*u_completo[e-1, 0]
    
    e+=1
print(R2x)

-224.99999999999997


In [35]:
e=1
R2y=0
while e<=6:
    R2y+=KG[3, e-1]*u_completo[e-1, 0]
    
    e+=1
print(R2y)

100.00000000000001


In [87]:
def tensao_e_deformacao2(n_elemento):
    E =  matriz_incidencia[n_elemento-1, 2]  
    
    no_1 = int(matriz_incidencia[n_elemento-1, 0])
    no_2 = int(matriz_incidencia[n_elemento-1, 1])
    
    # Pegar a coordenada do no_1
    x_no1 = matriz_nos[0, no_1-1]
    y_no1 = matriz_nos[1, no_1-1]
    
    # Pegar a coordenada do no_2
    x_no2 = matriz_nos[0, no_2-1]
    y_no2 = matriz_nos[1, no_2-1]
    
    l = sqrt((x_no2-x_no1)**2+(y_no2-y_no1)**2)
    
    c1 = [-1,1]
    c =np.array(c1)
    print(c)
    
    u1 = u[n_elemento,0]
    u0 = u[n_elemento-1,0]
    
    us = [u0,u1]
    
    ut = np.array(us).T
    print(ut)
    
    tensao = (E/l)  * np.dot(ut, c)
    print(E/l)
    print(np.dot(ut, c))
    return tensao

In [88]:
tensao_e_deformacao2(1)

[-1  1]
[-9.52380952e-07  1.60714286e-06]
525000000000.0
2.5595238095238095e-06


1343750.0

In [36]:
def tensao_e_deformacao(n_membro):
    
    matriz = np.array(([-1, 1]))
    if n_membro==1:
        
        matriz_aux=np.array(([0], [u[n_membro-1, 0]]))
        
        
    else:
        matriz_aux=np.array(([u[n_membro-2, 0]], [u[n_membro-1, 0]]))
       
    E =  matriz_incidencia[n_membro-1, 2]
    
    A = matriz_incidencia[n_membro-1, 3]
    
    
    no_1 = int(matriz_incidencia[n_membro-1, 0])
    no_2 = int(matriz_incidencia[n_membro-1, 1])
    
    # Pegar a coordenada do no_1
    x_no1 = matriz_nos[0, no_1-1]
    y_no1 = matriz_nos[1, no_1-1]
    
    # Pegar a coordenada do no_2
    x_no2 = matriz_nos[0, no_2-1]
    y_no2 = matriz_nos[1, no_2-1]
    
    l = sqrt((x_no2-x_no1)**2+(y_no2-y_no1)**2)
    
        
    tensao = E/l * np.dot(matriz, matriz_aux)
    
    deformacao = 1/l * np.dot(matriz, matriz_aux)
    
    return tensao, deformacao
        

In [37]:
e=1

while e<=num_membros:
    t, d = tensao_e_deformacao(e)
    print("tensão"+str(e)+": "+str(t)+" deformação"+str(e)+": "+str(d))
    
    e+=1

tensão1: [-500000.] deformação1: [-2.38095238e-06]
tensão2: [1791666.66666667] deformação2: [8.53174603e-06]
tensão3: [-2362500.] deformação3: [-1.125e-05]
